This driver has been tested with a FHR1000 spectrometer with one dual grating turret, an entrance slit, exit slit, and an exit slit selection mirror.

The dll unfortunately does not have any version information. It was shipped with LabSpec 6.5.1. The supporting information is contained in a word document called `SpeControl_dll.docx`.

In [1]:
from qcodes_contrib_drivers.drivers.Horiba.Horiba_FHR import HoribaFHR

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\Flash\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : C:\Users\Flash\.qcodes\logs\230718-6464-qcodes.log


In [2]:
dll_dir = r'C:\HORIBA\Software\SDK FHR Express'
spe = HoribaFHR(
    'spe', 
    port=13, 
    dll_dir=dll_dir,
    # This could be more sophisticated, like parsing an ini file ...
    grating_addrs=[5, 9],
    grating_kwargs=[
        {'label': '1800 gr/mm Grating',
         'min_value': 50e3,
         'max_value': 750e3,
         'coefficient_of_linearity': 1.000069196},
        {'label': '600 gr/mm Grating',
         'min_value': 50e3,
         'max_value': 2000e3,
         'coefficient_of_linearity': 0.999935906}
    ],
    slit_addrs=[1, 4],
    slit_kwargs=[
        {'label': 'Front entrance slit',
         'min_value': 750,
         'max_value': 2750},
        {'label': 'Side exit slit',
         'min_value': 700,
         'max_value': 2700}
    ],
    dc_addrs=[7],
    dc_kwargs=[
        {'label': 'Mirror',
         'val_mapping': {'front': 0, 'side': 1}}
    ]
)
spe.port.set_timeout(30000)

Following the 'manual' (`SpeControl_dll.docx` in the dll directory), the typical workflow would be to 
1. Set the grating ID
2. Initialize the grating
3. Setup the grating
4. Initialize the grating motor
5. Go to a given position

The instrument constructor already takes care of opening a connection and setting the correct baud rate.

In [3]:
grating_1800, grating_800 = spe.gratings

grating_1800.set_setup(min_speed=500, max_speed=280000, ramp=400, backlash=0, step=2, reverse=False)
# The help file doesn't say what specifically each phase does, so I assume they 
# are used for different driving levels of coarseness
grating_1800.set_ini_params(phase=1, min_speed=500, max_speed=280000, ramp=400)
grating_1800.set_ini_params(phase=2, min_speed=500, max_speed=13000, ramp=100)
grating_1800.set_ini_params(phase=3, min_speed=50, max_speed=600, ramp=50)
# Uncomment to actually initialize to zero-order
# grating_1800.init(54940)

grating_800.set_setup(min_speed=500, max_speed=280000, ramp=400, backlash=0, step=2, reverse=False)
grating_800.set_ini_params(phase=1, min_speed=500, max_speed=280000, ramp=400)
grating_800.set_ini_params(phase=2, min_speed=500, max_speed=13000, ramp=100)
grating_800.set_ini_params(phase=3, min_speed=50, max_speed=600, ramp=50)
# Uncomment to actually initialize to zero-order
# grating_800.init(2554965)

The slits are dealt with similarly, except they don't have different phases. 

According to the help file, they should allow for a step different from 1 (i.e., a pm unit instead of motor steps), but my device keeps running into timeout errors when I do so. The `init()` method hence initializes the slit to the completely closed state, and the values that `position.get()`/`position.set()` return or take are motor steps. In this particular case, the `slit_entrance.position(1000)` opens the front entrance slit by 250 motor steps, and `slit_entrance.position(1250)` by double that.

In [4]:
slit_entrance, slit_exit = spe.slits

slit_entrance.set_setup(min_speed=50, max_speed=500, ramp=150, backlash=0, step=1, reverse=True)
# Uncomment to actually initialize to completely close
# slit_entrance.init(750)

slit_exit.set_setup(min_speed=50, max_speed=500, ramp=150, backlash=0, step=1, reverse=True)
# Uncomment to actually initialize to completely close
# slit_exit.init(700)

The DC motors only have a settable binary position.

In [5]:
mirror, = spe.dcs
mirror.position('front')

In [ ]:
grating_800.init(2554965)
grating_800.position(800e3)  # pm
grating_800.position()

In [ ]:
slit_entrance.position(1000)
slit_entrance.position()

We can also set opening width, which corresponds to the position minus the zero-offset. If we hadn't initialized the entrance slit motor, we'd need to set the offset manually.

In [8]:
slit_entrance.offset = 750
slit_entrance.width(250)
slit_entrance.width()

250

In [9]:
spe.close()